### **Setup**

In [1]:
%%bash
pip install timm -q

### **Library Imports**

In [2]:
import os
import re
import cv2
import timm
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from time import time
from torch import nn, optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL
from torchvision import models, transforms

from sklearn.preprocessing import LabelEncoder

### **Utilities and Helpers**

In [3]:
SEED = 42
SIZE = 384
FOLD = 3
TRANSFORM = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize([0.49699, 0.58823, 0.23049],
                                                     [0.22591, 0.22614, 0.18264]),
                                ])
MODEL_PATH = f"../input/pdc-en2-a384f-f3/saves/ble_state.pt"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

le = LabelEncoder()


def breaker(num: int=50, char: str="*") -> None:
    print("\n" + num*char + "\n")


def get_image(path: str, size: int=224) -> np.ndarray:
    return cv2.resize(src=cv2.cvtColor(src=cv2.imread(path, cv2.IMREAD_COLOR), code=cv2.COLOR_BGR2RGB), dsize=(size, size), interpolation=cv2.INTER_AREA)

### **Configuration**

### **Dataset Template**

In [4]:
class DS(Dataset):
    def __init__(self, base_path: str, filenames: np.ndarray, size: int=224, transform=None):
        self.base_path = base_path
        self.filenames = filenames
        self.size = size
        self.transform = transform
    
    def __len__(self):
        return self.filenames.shape[0]
    
    def __getitem__(self, idx):
        return self.transform(get_image(os.path.join(self.base_path, self.filenames[idx]), size=self.size))

### **Model**

In [5]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.model = timm.create_model(model_name="efficientnet_b2", pretrained=False)
        # self.freeze()
        self.model.classifier = nn.Linear(in_features=self.model.classifier.in_features, out_features=10)
    
    def freeze(self):
        for params in self.parameters(): params.requires_grad = False

    def forward(self, x):
        return nn.LogSoftmax(dim=1)(self.model(x))

### **Predict Helpers**

In [6]:
def predict_batch(model=None, dataloader=None, path=None, device=None) -> np.ndarray:
    model.load_state_dict(torch.load(path, map_location=device)["model_state_dict"])
    model.to(device)    
    model.eval()
    
    y_pred = torch.zeros(1, 1).to(device)
    
    for X in dataloader:
        X = X.to(device)
        with torch.no_grad():
            output = torch.argmax(torch.exp(model(X)), dim=1)
        y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    
    return y_pred[1:].detach().cpu().numpy()

### **Submission**

In [7]:
df = pd.read_csv("../input/paddy-disease-classification/train.csv")
ss_df = pd.read_csv("../input/paddy-disease-classification/sample_submission.csv")

labels = df.label.copy().values
labels = le.fit_transform(labels)

ts_data_setup = DS(base_path="../input/paddy-disease-classification/test_images", 
                   filenames=ss_df.image_id.copy().values,
                   size=SIZE,
                   transform=TRANSFORM)
ts_data = DL(ts_data_setup, batch_size=64, shuffle=False)

model = Model().to(DEVICE)

y_pred = predict_batch(model=model, 
                       dataloader=ts_data, 
                       path=MODEL_PATH,
                       device=DEVICE)

ss_df["label"] = le.inverse_transform(y_pred.astype("uint8").reshape(-1))
ss_df.to_csv("submission.csv", index=False)